# Example: Regression with `QNNRegressor`

In this example we show how to train the `QNNRegressor` to solve a differential equation

## Imports

In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt


from squlearn import Executor
from squlearn.encoding_circuit import (
    KyriienkoEncodingCircuit,
)
from squlearn.encoding_circuit.layered_encoding_circuit import Layer
from squlearn.observables import SummedPaulis
from squlearn.qnn import QNNRegressor, ODELoss, get_lr_decay
from squlearn.optimizers import Adam

We will use the Pennylane quantum framework for executing the circuits.

In [2]:
executor = Executor("pennylane", shots=None)

## `QNNRegressor` Setup

We start by defining a parameterized quantum circuit (PQC), that implements $\ket{f_{\bm{\theta}, \phi}(x)} = U_{\bm{\theta}} U_{\phi}(x) \ket{0}$.

  $U_{\phi}(x)$ encodes the ODE information and $U_{\bm{\theta}}$ manipulates the function space.

In [3]:
nqubits = 6

circuit = KyriienkoEncodingCircuit(
    num_qubits=nqubits,
    encoding_style="chebyshev_tower",
    variational_arrangement="HEA",
    num_features=1,
    num_encoding_layers=1,
    num_variational_layers=5,
)

In [4]:
circuit.draw("mpl")

We also need an observable

In [5]:
observable = SummedPaulis(nqubits, include_identity=True)
print(observable)

SparsePauliOp(['IIIIII', 'IIIIIZ', 'IIIIZI', 'IIIZII', 'IIZIII', 'IZIIII', 'ZIIIII'],
              coeffs=[ParameterExpression(1.0*p[0]), ParameterExpression(1.0*p[1]),
 ParameterExpression(1.0*p[2]), ParameterExpression(1.0*p[3]),
 ParameterExpression(1.0*p[4]), ParameterExpression(1.0*p[5]),
 ParameterExpression(1.0*p[6])])



We create the homogeneous differential equation to be solved using a sympy symbolic expression


In [6]:
x = sp.symbols("x")
f, dfdx, dfdxdx = sp.symbols("f dfdx dfdxdx")
k = 0.1
lamb = 20
eq = lamb * sp.exp(-lamb * x * k) * sp.sin(lamb * x) + lamb * k * f + dfdx
initial_values = [1]

In [7]:
eq

dfdx + 2.0*f + 20*exp(-2.0*x)*sin(20*x)

Now we need to create an instance of `ODELoss`, the symbols involved in the ODE must be provided in the correct order: independent variable, function, and higher orders

In [8]:
loss_ODE_sp = ODELoss(
    eq,
    symbols_involved_in_ODE=[x, f, dfdx],
    initial_values=initial_values,
    boundary_handling="pinned",
)

Now we can create the `QNNRegressor` with the PQC and cost operator. We also specify `loss_ODE` and `Adam` as the loss function and optimizer respectively. 

In [9]:
param_obs = observable.generate_initial_parameters(seed=1)
param_ini = circuit.generate_initial_parameters(seed=1)


adam = Adam(options={"maxiter": 125, "tol": 0.00009, "lr": get_lr_decay(0.05, 0.02, 125)})

ode_regressor = QNNRegressor(
    circuit,
    observable,
    executor,
    loss_ODE_sp,
    adam,
    param_ini,
    param_obs,
    opt_param_op=True,
)

## Training

Now we are ready to train the model to solve the differential equation, (this takes time, around 120min)

In [10]:
x_space = np.linspace(0, 0.9, 15)
ref_values = np.zeros_like(x_space)
ode_regressor.fit(x_space.reshape(-1,1), ref_values)

fit:   0%|          | 0/125 [00:00<?, ?it/s]

ValueError: operand has more dimensions than subscripts given in einstein sum, but no '...' ellipsis provided to broadcast the extra dimensions.

## Evaluation

Let's check the trained parameters of the PQC and operator

In [ ]:
print("Result PQC params:", ode_regressor.param)
print("Result operator params:", ode_regressor.param_op)

Also let's plot the predicted function vs. the analytical solution

In [ ]:
x = np.linspace(0, 0.9, 15)
y = ode_regressor.predict(x)
plt.scatter(x, np.cos(20 * x) * np.exp(-20 * 0.1 * x))
plt.scatter(x, y)

x_span = np.linspace(0, 0.9, 100)
y_span = ode_regressor.predict(x_span)

plt.plot(x_span, np.cos(20 * x_span) * np.exp(-20 * 0.1 * x_span))
plt.plot(x_span, y_span)

plt.ylabel("f(x)")
plt.xlabel("x")